<a href="https://colab.research.google.com/github/hirdeshkumar2407/NLP_Group_Assigment/blob/main/1_word2vec_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word2Vec: Initial Setup

## Imports and stuff here:

In [23]:
!pip install --upgrade gensim


**After running the "!pip install --upgrade gensim"
 above, you MUST restart the session from the runtime options and then run the cell below**

In [24]:
# !pip install --upgrade gensim

import gensim
import re
import nltk
import string

from gensim.models.word2vec import Word2Vec
from nltk.tokenize import sent_tokenize, word_tokenize

import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## Loading the data from huggingface

In [25]:
import pandas as pd

df = pd.read_json("hf://datasets/FreedomIntelligence/RAG-Instruct/rag_instruct.json")

print(f"Total entries: {len(df)}")
df.info()


Total entries: 40541
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40541 entries, 0 to 40540
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   question   40541 non-null  object
 1   answer     40541 non-null  object
 2   documents  40541 non-null  object
dtypes: object(3)
memory usage: 950.3+ KB


## Word2vec Implementation

In [26]:
bulk_list = list(df["question"]) + list(df["answer"]) + list(df["documents"])
bulk_list = [subitem for item in bulk_list for subitem in (item if isinstance(item, list) else [item])]
sentences = []
for doc in bulk_list:
    # Step 1: Split into sentences
    if not doc:
        continue
    for sent in sent_tokenize(doc):
        # Step 2: Tokenize into words
        words = word_tokenize(sent)
        # Step 3: Remove punctuation
        words = [w for w in words if w not in string.punctuation]
        if words:  # ignore empty sentences
            sentences.append(words)



In [27]:
model = Word2Vec(sentences, vector_size=30, min_count=5, window=10)

In [28]:
term = 'AMD'
model.wv.most_similar(term)

[('Intel', 0.9741306304931641),
 ('Nvidia', 0.966727077960968),
 ('chipset', 0.9357510805130005),
 ('DEC', 0.9117414951324463),
 ('HP', 0.9105905294418335),
 ('ATI', 0.9085875749588013),
 ('ARM', 0.9019722938537598),
 ('IBM', 0.8989870548248291),
 ('GPU', 0.8948557376861572),
 ('Motorola', 0.8941123485565186)]

In [29]:
term = 'trump'
model.wv.most_similar(term)

[('five-card', 0.7952187657356262),
 ('dice', 0.7898284196853638),
 ('Players', 0.7889504432678223),
 ('doubles', 0.7811340689659119),
 ('tie', 0.7722302079200745),
 ('wins', 0.7710220813751221),
 ('contestant', 0.7673041820526123),
 ('ironing', 0.7644453644752502),
 ('contestants', 0.7329927086830139),
 ('Mickelson', 0.7321579456329346)]

In [30]:
#the classic one also works, nice!
vec = model.wv.get_vector('king') + (model.wv.get_vector('woman') - model.wv.get_vector('man'))
model.wv.similar_by_vector(vec)

[('king', 0.9438665509223938),
 ('emperor', 0.9332393407821655),
 ('throne', 0.9157471060752869),
 ('heir', 0.8885269165039062),
 ('prince', 0.8831648826599121),
 ('heirs', 0.8819757699966431),
 ('Emperor', 0.8783792853355408),
 ('bishop', 0.8683204054832458),
 ('duke', 0.8624470829963684),
 ('clan', 0.8594751358032227)]

## Using t-SNE to visualize the results of the vectorizatin

In [31]:
import random

sample = random.sample(list(model.wv.key_to_index), 500)
print(sample)

['twinned', 'Columns', 'GeV', 'Schadenfreude', 'Volksoper', 'Corp.', 'Dimensionality', 'Nagpur', 'K-1', 'Camillus', 'Fido', 'alien', 'odham', '10-carbon', 'Fluoromar', 'invitation', 'Aldeburgh', 'SemBioSys', 'Messrs.', 'Spice', 'KIC', 'Oryx', 'Ei-ichi', 'extruding', 'Garson', 'Tariffs', 'Camp', 'Bucherer', 'Michaelmas', 'Jango', 'fenestration', '1707', 'ESI', 'Cosell', 'tri', 'BPT', 'Obregón', 'Wasps', 'repurchase', 'goal-line', 'All-Stars', 'Splendor', 'antibodies', 'highlife', 'Dore', 'они', 'data-driven', 'internetworking', 'interdisciplinarity', 'Amasa', 'fax', 'rosea', 'interlacing', 'Saracens', 'Disasters', 'Powered', 'Hamilton–Jacobi–Bellman', 'stockbroker', 'arbitrarily', 'debridement', 'formula_22-paradoxical', '1458', 'iambic', 'revitalizing', 'Thousand', 'eventuate', 'Lincoln', 'Reed', 'Physician', '11,800', 'pé', 'brisk', 'ng/ml', 'JJ', 'hindbrain', 'opponent', 'Sébastien', 'Shelagh', 'DSSCs', '23-page', 'Orocobix', 'Preachers', '522', '.properties', 'denominator', 'dithiol

In [32]:
word_vectors = model.wv[sample]
word_vectors

array([[ 0.06788646, -0.58151215,  1.6003908 , ...,  0.6530722 ,
         0.21855655,  0.48474273],
       [ 0.08686287, -0.21170804,  0.14446826, ..., -0.2510813 ,
        -0.30747047, -0.32191983],
       [ 0.16631931, -0.13562016,  0.11810957, ...,  0.12669373,
        -0.04086367, -0.10859871],
       ...,
       [ 0.17922105, -0.17299412,  0.05449256, ...,  0.03204354,
        -0.21552657, -0.15693699],
       [ 0.22679651, -0.3295916 ,  0.02184317, ..., -0.02001583,
        -0.04169573, -0.13102615],
       [ 0.13662997, -0.31884512, -0.11099502, ..., -0.01502689,
        -0.08595143, -0.3557251 ]], dtype=float32)

In [33]:
word_vectors.shape

(500, 30)

In [34]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=3, n_iter=2000)
tsne_embedding = tsne.fit_transform(word_vectors)

/usr/local/lib/python3.11/dist-packages/sklearn/manifold/_t_sne.py:1164: FutureWarning:

'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.



In [35]:
import numpy as np

x, y, z = np.transpose(tsne_embedding)

In [36]:
!pip install plotly

In [37]:
import plotly.express as px

fig = px.scatter_3d(x=x, y=y, z=z)
fig.update_traces(marker=dict(size=3,line=dict(width=2)))
fig.show()

In [38]:
fig = px.scatter_3d(x=x[:200],y=y[:200],z=z[:200],text=sample[:200])
fig.update_traces(marker=dict(size=3,line=dict(width=2)),textfont_size=10)
fig.show()

In [39]:
# Add some specific terms to sample:
colours = ['red','green','blue','orange','yellow','purple','pink','cream','brown','black','white','gray']

word_vectors = model.wv[colours+sample]

tsne = TSNE(n_components=3)
tsne_embedding = tsne.fit_transform(word_vectors)

x, y, z = np.transpose(tsne_embedding)


import plotly.express as px

r = (-200,200)
fig = px.scatter_3d(x=x, y=y, z=z, range_x=r, range_y=r, range_z=r, text=colours + [None] * 500)
fig.update_traces(marker=dict(size=3,line=dict(width=2)),textfont_size=10)
fig.show()